In [4]:
#1a

import xml.etree.ElementTree as ET

def extract_bpmn_names(file_path):
    extracted_names = []
    
    try:
        # Load and parse the BPMN file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Iterate through all elements in the XML tree
        for elem in root.iter():
            # 1. Clean the tag: Remove the XML namespace (content inside {...})
            # e.g., '{http://www.omg.org/spec/BPMN/20100524/MODEL}task' becomes 'task'
            clean_tag = elem.tag.split('}')[-1]

            # 2. Check tag type based on your provided syntax
            # We look for: 'task' (generic), anything ending in 'Task', 'Gateway', or 'Event'
            is_task = (clean_tag == 'task' or clean_tag.endswith('Task'))
            is_gateway = clean_tag.endswith('Gateway')
            is_event = clean_tag.endswith('startEvent')

            if is_task or is_gateway or is_event:
                # 3. Extract the 'name' attribute
                # .get() returns None if the attribute doesn't exist (like in your StartEvent example)
                element_name = elem.get('name')
                
                # Only add if a name actually exists and isn't empty
                if element_name:
                    extracted_names.append(element_name)

        return extracted_names

    except FileNotFoundError:
        return ["Error: File not found."]
    except Exception as e:
        return [f"Error parsing file: {e}"]

# --- Usage Example ---
file_name = '/home/users3/asdf/s4423242/Downloads/passenger_security_cleaned.bpmn'  # Ensure this matches your file name
names = extract_bpmn_names(file_name)

print("--- Extracted Element Names ---")
for n in names:
    print(n)

--- Extracted Element Names ---
Passenger shows boarding pass
Passenger goes to security check
Passenger is checked
Suspicious?
manual control
Target destination in Schengen-Area?
Passenger goes to passport control
Passenger shows passport
Passenger goes to gate


In [5]:
#1b+c

import xml.etree.ElementTree as ET
import re
import os
from difflib import SequenceMatcher

# --- COMPONENT 1: String Matching ---
class StringMatchingComponent:
    def __init__(self, threshold=0.7):
        self.threshold = threshold

    def is_match(self, str1, str2):
        """
        Returns True if str1 and str2 are similar enough (fuzzy match).
        """
        # Exact match (case insensitive)
        if str1.lower() == str2.lower():
            return True
        
        # Fuzzy match
        ratio = SequenceMatcher(None, str1.lower(), str2.lower()).ratio()
        return ratio >= self.threshold

# --- COMPONENT 2: BPMN Analyzer (Extraction) ---
class BPMNAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.extracted_elements = {}  # {name: type}

    def parse_bpmn(self):
        """
        Parses the BPMN file to extract Tasks, Gateways, and Events.
        """
        if not os.path.exists(self.file_path):
            print(f"Error: File not found at {self.file_path}")
            return []

        try:
            tree = ET.parse(self.file_path)
            root = tree.getroot()

            found_names = []

            for elem in root.iter():
                clean_tag = elem.tag.split('}')[-1]

                # Check element types
                is_task = (clean_tag == 'task' or clean_tag.endswith('Task'))
                is_gateway = clean_tag.endswith('Gateway')
                is_event = clean_tag.endswith('Event') or clean_tag == 'startEvent'

                if is_task or is_gateway or is_event:
                    name = elem.get('name')
                    if name:
                        # Clean up newlines/spaces
                        clean_name = name.replace('\n', ' ').strip()
                        self.extracted_elements[clean_name] = clean_tag
                        found_names.append(clean_name)
            
            return found_names

        except Exception as e:
            print(f"Error parsing XML: {e}")
            return []

# --- COMPONENT 3: Precision/Recall Calculator ---
class MetricsCalculator:
    def __init__(self, ground_truth):
        self.ground_truth = ground_truth
        self.matcher = StringMatchingComponent(threshold=0.8)

    def calculate(self, extracted_elements):
        true_positives = []
        false_positives = []
        
        # We work on a copy of ground truth to track what we missed (False Negatives)
        remaining_ground_truth = self.ground_truth[:]

        # 1. Check every Extracted Element
        for extracted in extracted_elements:
            match_found = False
            matched_gt = None

            for gt in remaining_ground_truth:
                if self.matcher.is_match(extracted, gt):
                    match_found = True
                    matched_gt = gt
                    break
            
            if match_found:
                true_positives.append((extracted, matched_gt))
                # Remove from ground truth so it isn't matched twice
                remaining_ground_truth.remove(matched_gt)
            else:
                false_positives.append(extracted)

        # 2. What's left in Ground Truth are False Negatives (Missed)
        false_negatives = remaining_ground_truth

        # 3. Calculate Metrics
        tp = len(true_positives)
        fp = len(false_positives)
        fn = len(false_negatives)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        
        return {
            "precision": precision,
            "recall": recall,
            "tp_list": true_positives,
            "fp_list": false_positives,
            "fn_list": false_negatives
        }

# --- INPUTS ---
# 1. The Ground Truth provided in your prompt
GROUND_TRUTH = [
    "Passenger shows boarding pass",
    "Passenger goes to security check",
    "Passenger is checked",
    "Suspicious?",
    "manual control",
    "Target destination in Schengen-Area?",
    "Passenger goes to passport control",
    "Passenger shows passport",
    "Passenger goes to gate"
]

# 2. Path to your BPMN file
BPMN_PATH = '/home/users3/asdf/s4423242/Downloads/passenger_security_cleaned.bpmn'

# --- EXECUTION ---
print("--- 1. Extracting Elements from File ---")
analyzer = BPMNAnalyzer(BPMN_PATH)
extracted_names = analyzer.parse_bpmn()
print(f"Extracted {len(extracted_names)} elements from BPMN file.")

print("\n--- 2. Comparing against Ground Truth ---")
calculator = MetricsCalculator(GROUND_TRUTH)
metrics = calculator.calculate(extracted_names)

# --- OUTPUT REPORT ---
print(f"\nResults:")
print(f"Precision: {metrics['precision']:.2f} (How many extracted items were correct?)")
print(f"Recall:    {metrics['recall']:.2f} (How many ground truth items did we find?)")
print("-" * 60)

print("✅ True Positives (Correctly Extracted):")
for ext, gt in metrics['tp_list']:
    print(f"   '{ext}'")

print("\n❌ False Positives (Extracted but NOT in Ground Truth):")
for fp in metrics['fp_list']:
    print(f"   '{fp}'")

print("\n⚠️ False Negatives (In Ground Truth but NOT extracted):")
for fn in metrics['fn_list']:
    print(f"   '{fn}'")

--- 1. Extracting Elements from File ---
Extracted 9 elements from BPMN file.

--- 2. Comparing against Ground Truth ---

Results:
Precision: 1.00 (How many extracted items were correct?)
Recall:    1.00 (How many ground truth items did we find?)
------------------------------------------------------------
✅ True Positives (Correctly Extracted):
   'Passenger shows boarding pass'
   'Passenger goes to security check'
   'Passenger is checked'
   'Suspicious?'
   'manual control'
   'Target destination in Schengen-Area?'
   'Passenger goes to passport control'
   'Passenger shows passport'
   'Passenger goes to gate'

❌ False Positives (Extracted but NOT in Ground Truth):

⚠️ False Negatives (In Ground Truth but NOT extracted):


In [6]:
import json
import re
from difflib import SequenceMatcher
from typing import List

# --- Haystack Imports ---
from haystack.dataclasses import ChatMessage
from haystack.components.builders import ChatPromptBuilder
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

# ==========================================
# 1. EVALUATION COMPONENT (Precision/Recall)
# ==========================================
class BPMNEvaluator:
    def __init__(self, threshold=0.8):
        self.threshold = threshold

    def _is_match(self, str1, str2):
        # Returns True if similarity is above threshold
        return SequenceMatcher(None, str1.lower(), str2.lower()).ratio() >= self.threshold

    def calculate_metrics(self, ground_truth_list, extracted_list):
        true_positives = []
        false_positives = []
        # Copy ground truth to track what we miss (False Negatives)
        remaining_gt = ground_truth_list[:]

        for extracted in extracted_list:
            match_found = False
            matched_gt = None
            
            # Check if extracted item matches any item in ground truth
            for gt in remaining_gt:
                if self._is_match(extracted, gt):
                    match_found = True
                    matched_gt = gt
                    break
            
            if match_found:
                true_positives.append(extracted)
                remaining_gt.remove(matched_gt) # Don't match the same GT item twice
            else:
                false_positives.append(extracted)

        tp = len(true_positives)
        fp = len(false_positives)
        fn = len(remaining_gt)

        # Avoid DivisionByZero errors
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        return {
            "precision": round(precision, 2),
            "recall": round(recall, 2),
            "tp": true_positives,
            "fp": false_positives,
            "fn": remaining_gt
        }

# ==========================================
# 2. DATA SETUP (3 Cases)
# ==========================================
test_cases = [
    {
        "name": "Case 1: Passenger Security",
        "text": """
        First, the Passenger shows boarding pass. Then the Passenger goes to security check.
        The officer decides if the passenger is Suspicious?
        If yes, the Passenger is checked manually.
        Finally, the Passenger goes to the gate.
        """,
        "ground_truth": [
            "Passenger shows boarding pass", "Passenger goes to security check", 
            "Suspicious?", "Passenger is checked manually", "Passenger goes to gate"
        ]
    },
    {
        "name": "Case 2: Order Processing",
        "text": """
        An Order is received. The system Checks stock availability.
        If stock is available?, the order is Shipped.
        If not, an Error Email is sent.
        The process ends when the order is fulfilled.
        """,
        "ground_truth": [
            "Order is received", "Checks stock availability", "stock is available?", 
            "Shipped", "Error Email is sent", "order is fulfilled"
        ]
    },
    {
        "name": "Case 3: Loan Application",
        "text": """
        The client submits a loan application. The bank Assess credit risk.
        Is the risk acceptable? If yes, Approve Loan.
        If no, Reject Loan.
        """,
        "ground_truth": [
            "submits a loan application", "Assess credit risk", 
            "Is the risk acceptable?", "Approve Loan", "Reject Loan"
        ]
    }
]

# ==========================================
# 3. PIPELINE DEFINITION
# ==========================================

# Define the JSON Schema for the LLM
json_schema = json.dumps({
    "tasks": ["task_name_1", "task_name_2"],
    "gateways": ["gateway_name"],
    "events": ["event_name"]
})

# Define the Prompt Template
template_text = """
Extract all **task names**, **gateways** and **events** from the BPMN description provided in the passage: {{passage}}.

Return the extracted elements as a single JSON object that adheres strictly to the following schema:
{{json_schema}}

Only return the JSON. Do not include markdown formatting like ```json.
"""

# Initialize Haystack Components
# We wrap the string in ChatMessage.from_user
prompt_builder = ChatPromptBuilder(template=[ChatMessage.from_user(template_text)],required_variables=["passage", "json_schema"]
)

# --- FIX APPLIED HERE ---
# 1. URL changed to "http://localhost:11434" (No /api/generate)
# 2. Model changed to "llama3" (Ensure you have run 'ollama pull llama3')
llm = OllamaChatGenerator(
    model="llama3.1.8:b", 
    url="http://localhost:11434",
    generation_kwargs={
        "format": "json" # Forces Ollama to output valid JSON
    }
)

# ==========================================
# 4. RUNNING THE PIPELINE LOOP (DEBUG VERSION)
# ==========================================
evaluator = BPMNEvaluator(threshold=0.7)

print(f"{'TEST CASE':<30} | {'PRECISION':<10} | {'RECALL':<10}")
print("-" * 60)

for case in test_cases:
    # 1. Build Prompt
    res = prompt_builder.run(
        passage=case["text"], 
        json_schema=json_schema
    )
    
    try:
        # --- THE PROBLEMATIC LINE ---
        # We try to run it, but catch errors if it fails
        result = llm.run(messages=res["prompt"])
        response_text = result["replies"][0].content
        
        # 3. Parse JSON
        # Look for JSON content between braces {} to ignore extra text
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if json_match:
            clean_json = json_match.group(0)
            data = json.loads(clean_json)
        else:
            clean_json = response_text.replace("```json", "").replace("```", "").strip()
            data = json.loads(clean_json)
            
        # Flatten dictionary for evaluation
        extracted_flat = []
        for key in ["tasks", "gateways", "events"]:
            extracted_flat.extend(data.get(key, []))
            
    except Exception as e:
        # --- ERROR HANDLER ---
        print(f"\n[!] CRITICAL ERROR on {case['name']}:")
        print(f"    Reason: {e}")
        print("    Suggestion: Check if Ollama is running and if you pulled the model.")
        print("    Command to fix: 'ollama run llama3'")
        extracted_flat = []

    # 4. Evaluate
    metrics = evaluator.calculate_metrics(case["ground_truth"], extracted_flat)
    
    # 5. Print Row
    print(f"{case['name']:<30} | {metrics['precision']:<10} | {metrics['recall']:<10}")

TEST CASE                      | PRECISION  | RECALL    
------------------------------------------------------------

[!] CRITICAL ERROR on Case 1: Passenger Security:
    Reason: model "llama3.1.8:b" not found, try pulling it first (status code: 404)
    Suggestion: Check if Ollama is running and if you pulled the model.
    Command to fix: 'ollama run llama3'
Case 1: Passenger Security     | 0.0        | 0.0       

[!] CRITICAL ERROR on Case 2: Order Processing:
    Reason: model "llama3.1.8:b" not found, try pulling it first (status code: 404)
    Suggestion: Check if Ollama is running and if you pulled the model.
    Command to fix: 'ollama run llama3'
Case 2: Order Processing       | 0.0        | 0.0       

[!] CRITICAL ERROR on Case 3: Loan Application:
    Reason: model "llama3.1.8:b" not found, try pulling it first (status code: 404)
    Suggestion: Check if Ollama is running and if you pulled the model.
    Command to fix: 'ollama run llama3'
Case 3: Loan Application       